In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget

import pandas as pd
import xarray as xr

In [2]:
ts = xr.open_dataset('HEAVY--Land_and_Ocean_LatLong1.nc')

In [3]:
ts

<xarray.Dataset>
Dimensions:      (longitude: 360, latitude: 180, time: 2078, month_number: 12)
Coordinates:
  * longitude    (longitude) float32 -179.5 -178.5 -177.5 ... 177.5 178.5 179.5
  * latitude     (latitude) float32 -89.5 -88.5 -87.5 -86.5 ... 87.5 88.5 89.5
  * time         (time) float64 1.85e+03 1.85e+03 ... 2.023e+03 2.023e+03
Dimensions without coordinates: month_number
Data variables:
    land_mask    (latitude, longitude) float64 ...
    temperature  (time, latitude, longitude) float32 ...
    climatology  (month_number, latitude, longitude) float32 ...
Attributes:
    Conventions:           Berkeley Earth Internal Convention (based on CF-1.5)
    title:                 Native Format Berkeley Earth Surface Temperature A...
    history:               22-Mar-2023 06:54:41
    institution:           Berkeley Earth Surface Temperature Project
    land_source_history:   05-Mar-2023 08:24:35
    ocean_source_history:  22-Mar-2023 05:21:28
    comment:               This file contains Berkeley Earth surface temperat...